<a href="https://colab.research.google.com/github/returaj/cs6910/blob/assginment1_akash/q10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/DLASSIGNMENT/activation_func.py .
import activation_func
!cp drive/MyDrive/DLASSIGNMENT/loss_func.py .
import loss_func
!cp drive/MyDrive/DLASSIGNMENT/runner.py .
!cp drive/MyDrive/DLASSIGNMENT/feed_forward_nn.py .
!cp drive/MyDrive/DLASSIGNMENT/measure.py .
!cp drive/MyDrive/DLASSIGNMENT/optimizer.py .


Mounted at /content/drive


In [3]:
import numpy as np
from keras.datasets import mnist

from feed_forward_nn import FNN
from measure import accuracy
from optimizer import SGD, MomentumGD, NesterovGD, Rmsprop, Adam, Nadam
from activation_func import Sigmoid, Relu, Tanh
from loss_func import CrossEntropy, MeanSquaredError
from sklearn.model_selection import train_test_split
import numpy as np
from keras.datasets import fashion_mnist
from feed_forward_nn import FNN
from measure import accuracy
from optimizer import SGD, MomentumGD, NesterovGD, Rmsprop, Adam, Nadam
from activation_func import Sigmoid, Relu, Tanh
from loss_func import CrossEntropy, MeanSquaredError
from sklearn.model_selection import train_test_split
#np.random.seed(2)


In [12]:
class digits_mnist(object):
  def __init__(self, MnistData=True):
    if MnistData:
      self.initialize_mnist_data()
    else:
      self.initialize_data()

  def initialize_mnist_data(self):
    (X, self.y), (X_test, self.y_test) = mnist.load_data()
    self.X = np.array([x.flatten() for x in X]) / 255
    self.X_test = np.array([x.flatten() for x in X_test]) / 255

  def initialize_data(self):
    raise NotImplementedError("Please implement this method if you need other dataset.")

  @staticmethod
  def get_loss_function(key):
    mapper = {
      "cross_entropy": CrossEntropy,
      "mse": MeanSquaredError
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def get_activation_function(key):
    mapper = {
      "sigmoid": Sigmoid,
      "relu": Relu,
      "tanh": Tanh
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def get_optimizer(key):
    mapper = {
      "sgd": SGD,
      "momentum_gd": MomentumGD,
      "nesterov_gd": NesterovGD,
      "rmsprop": Rmsprop,
      "adam": Adam,
      "nadam": Nadam
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def train(X_train, y_train, params,id,do_val):
    """
    X_train: (batch_size(B), data_size(N))
    y_train: (batch_size(B))
    params: dict(
      batch_size: int,
      epochs: int,
      alpha: float,
      optimizer: One of (SGD, MomentumGD, NesterovGD, Rmsprop, Adam, Nadam),
      hidden_layers_size: list(layer_size),
      act_func: One of (Sigmoid, Relu, Tanh),
      reg: float,
      init: One of (random, xavier),
      loss_func: One of (CrossEntropy, MeanSquaredError)
    )
    """
    print(f"model:{id}")
    if do_val:
      X_train, X_val, y_train, y_val = train_test_split(
          X_train, y_train, test_size=0.1, random_state=10)

    data_size, input_size = X_train.shape
    output_size = 10
    batch_size, epochs = params['batch_size'], params['epochs']
    act_func, loss_func = params['act_func'](), params['loss_func']()
    model = FNN(
      input_size         = input_size,
      output_size        = output_size,
      hidden_layers_size = params['hidden_layers_size'],
      act_func           = act_func,
      reg                = params['reg'],
      init               = params['init'],
      loss_func          = loss_func,
    )
    optimizer = params['optimizer'](model, params['alpha'])
    for ep in range(1, epochs+1):
      ids = np.arange(data_size)
      np.random.shuffle(ids)
      start, end = 0, batch_size
      while end > start:
        x, y = X_train[ids[start:end]], y_train[ids[start:end]]
        optimizer.optimize(x, y)
        start, end = end, min(data_size, end+batch_size)
      # log
      train_loss = loss_func.error(X_train, y_train, model)
      estimate_y_train = digits_mnist.predict(X_train, model)
      train_acc = accuracy(estimate_y_train, y_train)
      val_loss, val_acc = "NotDefined", "NotDefined"
      if do_val:
        val_loss = loss_func.error(X_val, y_val, model)
        estimate_y_val = digits_mnist.predict(X_val, model)
        val_acc = accuracy(estimate_y_val, y_val)
    digits_mnist.logger(train_loss, train_acc, val_loss, val_acc)
    return model

  def logger(train_loss, train_acc, val_loss, val_acc):
    #print(f"epochs:{epoch}")
    print(f"TrainingLoss: {train_loss}, TrainingAccuracy: {train_acc}")
    print(f"ValidationLoss: {val_loss}, ValidationAccuracy: {val_acc}")

  @staticmethod
  def predict(X, model):
    """
    X: (batch_size(B), data_size(N))
    return np.array of size: (batch_size(B),)
    """
    prob = model.forward(X)[-1]
    return np.argmax(prob, axis=1)


#WANDB_PROJECT = "Confusion matrix"
#WANDB_ENTITY  = "cs21m003_cs21d406"


def run(epochs,batch_size,hidden_layers,hidden_nodes,learning_rate,optimizer,act_func,reg,weight_init,id,loss_func):
  #wandb.init(project=WANDB_PROJECT, entity=WANDB_ENTITY)
  #config = wandb.config
  loss_name = "ce" if loss_func == "cross_entropy" else "mse"
  #wandb.run.name=f"e_{config.epochs}_bs_{config.batch_size}_hl_{config.hidden_layers}_hn_{config.hidden_nodes}_init_{config.weight_init}_ac_{config.act_func}_reg_{config.reg}_ls_{loss_name}_opt_{config.optimizer}_lr_{config.learning_rate}_sc_{config.search_type}_best"
  hidden_layers_size = [hidden_nodes] * hidden_layers
  runner = digits_mnist()
  params = {
    "batch_size"        : batch_size,
    "epochs"            : epochs,
    "alpha"             : learning_rate,
    "optimizer"         : runner.get_optimizer(optimizer),
    "hidden_layers_size": hidden_layers_size,
    "act_func"          : runner.get_activation_function(act_func),
    "reg"               : reg,
    "init"              : weight_init,
    "loss_func"         : runner.get_loss_function(loss_func),
  }
  model = runner.train(runner.X, runner.y, params,id,do_val=True)
  estimate_y_test = runner.predict(runner.X_test, model)
  test_acc = accuracy(estimate_y_test, runner.y_test)
  print(f'test_accuracy:{test_acc}')




if __name__ == '__main__':
  run(5,32,3,128,0.001,"adam","tanh",0,'xavier',1,"cross_entropy")
  run(5,64,3,128,0.001,"adam","tanh",0,'xavier',2,"cross_entropy")
  run(5,32,3,32,0.001,"nadam","tanh",0.0005,'xavier',3,"cross_entropy")


model:1
TrainingLoss: 0.05407586550235132, TrainingAccuracy: 0.9832962962962963
ValidationLoss: 0.1184817148094781, ValidationAccuracy: 0.9676666666666667
test_accuracy:0.9686
model:2
TrainingLoss: 0.056299638259284614, TrainingAccuracy: 0.9829814814814815
ValidationLoss: 0.10671999185562739, ValidationAccuracy: 0.9703333333333334
test_accuracy:0.9664
model:3
TrainingLoss: 0.20367736748791548, TrainingAccuracy: 0.9708148148148148
ValidationLoss: 0.2392556843479847, ValidationAccuracy: 0.9606666666666667
test_accuracy:0.9602
